In [1]:
import pandas as pd
import csv
from pathlib import Path
import json
import numpy as np
from datetime import datetime
import time
import os

In [10]:
root_path = #deleted for security reasons
out_path = #deleted for security reasons

In [3]:
# "data" are all sheets as a dictionary
data = pd.read_excel(root_path/'cofacts_20220322-20220422_20220503dump.xlsx', sheet_name = None) 
xls = pd.ExcelFile(root_path/'cofacts_20220322-20220422_20220503dump.xlsx')

/opt/homebrew/lib/python3.9/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [4]:
# see all sheet names
xls.sheet_names

['WebUser', 'WebVisit', 'lineUser', 'lineVisit', 'Reply']

In [6]:
webuser = set(data.get('WebUser')['id'])
webwebvisituser = set(data.get('WebVisit')['id'])
lineuser = set(data.get('lineUser')['id'])
linevisit = set(data.get('lineVisit')['id'])
reply = data.get('Reply')

In [7]:
#總回報量
all_article_id = webuser.union(webwebvisituser).union(lineuser).union(linevisit)
print(len(all_article_id))

16892


In [8]:
reply = data.get('Reply')

In [9]:
for c in reply.columns:
    reply = reply.rename(columns={c:c.replace(' ','_')})

In [11]:
#已查核筆數
replied_article_id_set = set(reply['article_id'])
print(len(replied_article_id_set))

9496


In [168]:
#未查核筆數
len(all_article_id) - len(replied_article_id_set)

7396

In [13]:
#被查證比例
len(replied_article_id_set)/len(all_article_id)

0.5621596021785461

In [15]:
#單篇回報被查證數量
reply_count = reply[['article_id', 'reply_id']].groupby(by='article_id', as_index=False).count()\
.sort_values(by='reply_id', ascending=False)\
.rename(columns={'reply_id':'reply_id_size'})

# 編碼

In [19]:
# {'NOT_ARTICLE', 'NOT_RUMOR', 'OPINIONATED', 'RUMOR'}
# 不在查證範圍, 含有真實訊息, 含有個人意見, 含有錯誤訊息
start = time.process_time()
end = time.process_time()

for rows in reply.itertuples():
    if reply.loc[rows.Index, 'reply_type'] == 'RUMOR':
        reply.loc[rows.Index, 'type_score'] = -1
    elif reply.loc[rows.Index, 'reply_type'] == 'NOT_RUMOR':
        reply.loc[rows.Index, 'type_score'] = 1
    elif reply.loc[rows.Index, 'reply_type'] == 'OPINIONATED':
        reply.loc[rows.Index, 'type_score'] = -1
    elif reply.loc[rows.Index, 'reply_type'] == 'NOT_ARTICLE':
        reply.loc[rows.Index, 'type_score'] = 0
        
print("This time is being calculated")
print(end - start)  

This time is being calculated
4.800000000315663e-05


# article_truth_score
## count

In [22]:
#每個回應給出的truth_score
start = time.process_time()
end = time.process_time()

for rows in reply.itertuples():
    reply.loc[rows.Index, 'reply_truth_score'] \
        =reply.loc[rows.Index, 'type_score']*reply.loc[rows.Index, 'reply_positiveFeedbackCount']\
        -reply.loc[rows.Index, 'type_score']*reply.loc[rows.Index, 'reply_negativeFeedbackCount']

print("This time is being calculated")
print(end - start)  

This time is being calculated
4.900000000063187e-05


In [26]:
#相加成每篇文章的truth_score
reply_article_truth_score = reply[['article_id', 'reply_truth_score']].groupby(by='article_id', as_index=False).sum()\
                            .rename(columns={'reply_truth_score':'article_truth_score_count'})

In [104]:
reply_type_count=[]

for idx in replied_article_id_set:
    reply_type_set = set(reply[reply.article_id==idx]['reply_type'])
    reply_type_count.append((idx, len(reply_type_set), str(reply_type_set)))
    
article_reply_type_set = pd.DataFrame(reply_type_count, 
                                      columns=['article_id', 'reply_type_count', 'reply_type_set'])

In [105]:
reply_size = reply[['article_id', 'reply_id']].groupby(by='article_id', as_index=False).size()

In [114]:
article_truth_score_count = article_reply_type_set.merge(reply_size).merge(reply_article_truth_score)

In [116]:
for rows in article_truth_score_count.itertuples():
    if article_truth_score_count.loc[rows.Index, 'article_truth_score_count'] >0:
        article_truth_score_count.loc[rows.Index, 'article_type_count'] = '不是謠言'
    elif article_truth_score_count.loc[rows.Index, 'article_truth_score_count'] ==0:
        article_truth_score_count.loc[rows.Index, 'article_type_count'] = '無法判斷或不在查證範圍'
    elif article_truth_score_count.loc[rows.Index, 'article_truth_score_count'] <0:
        article_truth_score_count.loc[rows.Index, 'article_type_count'] = '謠言或個人意見'

In [117]:
article_truth_score_count.sort_values(by=['article_truth_score_count', 'reply_type_count'], ascending=True)

,article_id,reply_type_count,reply_type_set,size,article_truth_score_count,article_type_count
7155,3uzlh41rh4oqm,1,{'RUMOR'},2,-1006.0,謠言或個人意見
6735,1qhjmled1mv2w,1,{'RUMOR'},2,-502.0,謠言或個人意見
6058,oikfzsvt011y,2,"{'NOT_ARTICLE', 'RUMOR'}",4,-310.0,謠言或個人意見
7670,119qngktbf3wm,1,{'RUMOR'},3,-277.0,謠言或個人意見
3059,m8ygcym6vxz1,1,{'RUMOR'},1,-240.0,謠言或個人意見
...,...,...,...,...,...,...
6775,1wde3u5mvcqzy,1,{'NOT_RUMOR'},1,77.0,不是謠言
1397,2lnupn6kprnz5,1,{'NOT_RUMOR'},1,79.0,不是謠言
2547,3ppeiqj59yffi,1,{'NOT_RUMOR'},1,111.0,不是謠言
4531,1frt07jdgygwy,1,{'NOT_RUMOR'},1,112.0,不是謠言


In [31]:
article_truth_score.to_csv(out_path/'cofacts_truth_score_20220322-20220422.csv', index=False)

## normalize

In [79]:
#相同reply_type的like/dislike加總
reply_type_prop = reply[['article_id', 'reply_type', 'type_score', 
                         'reply_positiveFeedbackCount', 'reply_negativeFeedbackCount']]\
                  .groupby(by=['article_id', 'reply_type', 'type_score'], as_index=False).sum()

In [68]:
reply[['article_id', 'reply_type', 'type_score', 
        'reply_positiveFeedbackCount', 'reply_negativeFeedbackCount']].shape

(10951, 5)

In [69]:
reply_type_like_count = reply_type_prop[['article_id', 'reply_positiveFeedbackCount', 'reply_negativeFeedbackCount']]\
                        .groupby(by='article_id', as_index=False).sum()

In [75]:
reply_type_like_count.shape

(9496, 4)

In [70]:
reply_type_like_count['article_denominator'] = reply_type_like_count['reply_positiveFeedbackCount']\
                                     + reply_type_like_count['reply_negativeFeedbackCount']

In [90]:
reply_type_prop['numerator'] = (reply_type_prop['reply_positiveFeedbackCount']\
                                -reply_type_prop['reply_negativeFeedbackCount'])*reply_type_prop['type_score']

In [93]:
reply_type_prop_numerator = reply_type_prop[['article_id', 'numerator']].groupby(by='article_id', as_index=False).sum()

In [95]:
reply_type_prop_fraction = reply_type_prop_numerator.merge(reply_type_like_count)

In [96]:
reply_type_prop_fraction['reply_type_prop'] = reply_type_prop_fraction['numerator']/reply_type_prop_fraction['article_denominator']

In [102]:
reply_type_prop_fraction['reply_type_prop'] = reply_type_prop_fraction['reply_type_prop'].fillna(0)

In [118]:
for rows in reply_type_prop_fraction.itertuples():
    if reply_type_prop_fraction.loc[rows.Index, 'reply_type_prop'] >0:
        reply_type_prop_fraction.loc[rows.Index, 'article_type_prop'] = '不是謠言'
    elif reply_type_prop_fraction.loc[rows.Index, 'reply_type_prop'] ==0:
        reply_type_prop_fraction.loc[rows.Index, 'article_type_prop'] = '無法判斷或不在查證範圍'
    elif reply_type_prop_fraction.loc[rows.Index, 'reply_type_prop'] <0:
        reply_type_prop_fraction.loc[rows.Index, 'article_type_prop'] = '謠言或個人意見'

In [129]:
article_truth_score_merge = article_truth_score_count.merge(reply_type_prop_fraction)\
                            .drop(columns=['size', 'reply_positiveFeedbackCount', 'reply_negativeFeedbackCount',
                                           'numerator','article_denominator'])

In [132]:
article_truth_score_merge.reindex(columns=['article_id', 'reply_type_count', 'reply_type_set',
       'article_truth_score_count', 'article_type_count', 'reply_type_prop', 'article_type_prop'])

,article_id,reply_type_count,reply_type_set,article_truth_score_count,article_type_count,reply_type_prop,article_type_prop
0,1dqqhpqxn4dk0,1,{'RUMOR'},-1.0,謠言或個人意見,-0.200000,謠言或個人意見
1,3t48ii3p66cbm,1,{'OPINIONATED'},-2.0,謠言或個人意見,-1.000000,謠言或個人意見
2,1bx9gnebv6p01,1,{'RUMOR'},-2.0,謠言或個人意見,-1.000000,謠言或個人意見
3,14iyzlbojkvut,1,{'OPINIONATED'},-2.0,謠言或個人意見,-1.000000,謠言或個人意見
4,2a2si2nph7960,1,{'OPINIONATED'},-2.0,謠言或個人意見,-1.000000,謠言或個人意見
...,...,...,...,...,...,...,...
9491,371hkwndfzqp8,1,{'RUMOR'},-1.0,謠言或個人意見,-1.000000,謠言或個人意見
9492,21g65qh6pls6e,1,{'RUMOR'},-5.0,謠言或個人意見,-1.000000,謠言或個人意見
9493,3e35eai9dxjf8,1,{'NOT_RUMOR'},1.0,不是謠言,1.000000,不是謠言
9494,53xxwz5xb9ei,1,{'NOT_RUMOR'},2.0,不是謠言,1.000000,不是謠言


# 檢查兩種方式的結果是否相同

In [141]:
for rows in article_truth_score_merge.itertuples():
    if article_truth_score_merge.loc[rows.Index, 'article_type_count'] != article_truth_score_merge.loc[rows.Index, 'article_type_prop']:
        article_truth_score_merge.loc[rows.Index, 'comp'] = 'inconsistent'
    else:
        article_truth_score_merge.loc[rows.Index, 'comp'] = 'consistent'

In [143]:
set(article_truth_score_merge.comp)

{'consistent'}

In [147]:
article_truth_score_merge.to_csv(out_path/'cofacts_truth_score_merge_20220322-20220422.csv', index=False)

In [175]:
article_truth_score_merge[['article_type_prop', 'article_id']].groupby(by='article_type_prop', as_index=False).count()

,article_type_prop,article_id
0,不是謠言,1625
1,無法判斷或不在查證範圍,863
2,謠言或個人意見,7008
